# Disable of ETW Trace
Detects a command that clears or disables any ETW trace log which could indicate a logging evasion.

## Rule Content
```
- title: Disable of ETW Trace
  id: a238b5d0-ce2d-4414-a676-7a531b3d13d6
  description: Detects a command that clears or disables any ETW trace log which could
    indicate a logging evasion.
  references:
  - https://docs.microsoft.com/en-us/windows-server/administration/windows-commands/wevtutil
  - https://github.com/Neo23x0/sigma/blob/master/rules/windows/process_creation/win_mal_lockergoga.yml
  - https://abuse.io/lockergoga.txt
  author: '@neu5ron, Florian Roth'
  date: 2019/03/22
  tags:
  - attack.execution
  - attack.t1070
  - car.2016-04-002
  level: high
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection_clear_1:
      CommandLine: '* cl */Trace*'
    selection_clear_2:
      CommandLine: '* clear-log */Trace*'
    selection_disable_1:
      CommandLine: '* sl* /e:false*'
    selection_disable_2:
      CommandLine: '* set-log* /e:false*'
    condition: selection_clear_1 or selection_clear_2 or selection_disable_1 or selection_disable_2

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:*\ cl\ *\/Trace* OR process_command_line.keyword:*\ clear\-log\ *\/Trace* OR process_command_line.keyword:*\ sl*\ \/e\:false* OR process_command_line.keyword:*\ set\-log*\ \/e\:false*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()